In [7]:
!pip install pymorphy2 tqdm  

In [9]:
import re, numpy as np, pandas as pd  # заранее проведем все необходимые импорты
from tqdm import tqdm
import pymorphy2
from functools import lru_cache
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support, confusion_matrix
import warnings

warnings.filterwarnings('ignore')

In [29]:
# загрузка данных

train_path = "data/train(1).csv"
test_path  = "data/test(1).csv"
data_path  = "data/data(3).csv"

train = pd.read_csv(train_path, sep=';')
test  = pd.read_csv(test_path, sep=';')
data  = pd.read_csv(data_path, sep=';')

In [19]:
import inspect  # функция, помогающая избежать проблем с модулем getargspec в pymorphy (иногда простое обновление библиотек не помогает)
if not hasattr(inspect, 'getargspec'):
    from collections import namedtuple
    ArgSpec = namedtuple('ArgSpec', 'args varargs keywords defaults')
    def getargspec(func):
        spec = inspect.getfullargspec(func)
        return ArgSpec(spec.args, spec.varargs, spec.varkw, spec.defaults)
    inspect.getargspec = getargspec

In [23]:
tqdm.pandas()

morph = pymorphy2.MorphAnalyzer()

@lru_cache(maxsize=200000)  # используем очистку кэша, так как данных довольно много и они могут долго обрабатываться
def lemma(tok: str) -> str:
    return morph.parse(tok)[0].normal_form

def clean(s: str) -> str: # проведем минимальную предобработку текста: приведем к нижнему регистру, избавимся от пропусков и спецзнаков, проведем лемматизацию
    s = str(s).lower()
    s = re.sub(r'https?://\S+|www\.\S+', ' ', s)
    s = re.sub(r'[^a-zа-яё0-9?\!\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def lemmatize(s: str) -> str:
    return " ".join(lemma(t) for t in s.split()) if s else ""

# объединим данные из train и test по ID с соовтетствующими вопрсоами из data: так мы получим метки классов для train
train_full = train.merge(data, on="ID", how="left")   
test_full  = test.merge(data,  on="ID", how="left")


train_full["question_raw"] = train_full["Question"].fillna("").map(clean)
test_full["question_raw"]  = test_full["Question"].fillna("").map(clean)

train_full["question_lem"] = train_full["question_raw"].progress_map(lemmatize)
test_full["question_lem"]  = test_full["question_raw"].progress_map(lemmatize)

X_all_df = train_full[["question_lem","question_raw"]]  # готовые вопросы для обучения алгоритмов
y = train_full["Answer"].astype(int).values

X_tr_df, X_va_df, y_tr, y_va = train_test_split(
    X_all_df, y, test_size=0.2, stratify=y, random_state=42
)

tfidf_word = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2, max_features=150_000) # векторизируем вопросы
tfidf_char = TfidfVectorizer(analyzer='char', ngram_range=(3,6), min_df=2, max_features=120_000)

ct = ColumnTransformer(
    transformers=[
        ("w", tfidf_word, "question_lem"),
        ("c", tfidf_char, "question_raw"),
    ],
    sparse_threshold=1.0
)  #используем ColumnTransformer для упрощения работы с разными фичами, чтобы избежать дополнительных циклов и функций

X_tr_tfidf = ct.fit_transform(X_tr_df)
X_va_tfidf = ct.transform(X_va_df)


# добавим мета-фичи: длина вопроса, количество вопросительных знаков, восклицательных знаков, цифр
def make_meta(series: pd.Series):
    return np.vstack([
        series.str.len(),
        series.str.count(r'\?'),
        series.str.count(r'!'),
        series.str.count(r'\d'),
    ]).T

meta_tr = csr_matrix(make_meta(X_tr_df["question_raw"]))
meta_va = csr_matrix(make_meta(X_va_df["question_raw"]))

X_tr = hstack([X_tr_tfidf, meta_tr], format='csr')  #соединяем все полученные признаки в датасеты
X_va = hstack([X_va_tfidf, meta_va], format='csr')


# так как у нас явный дисбаланс классов, то необходимо добавить бОльшие веса миноритарному классу
n_pos = int((y_tr==1).sum()); n_neg = int((y_tr==0).sum()) # считаем количество объектов мажоританого и миноритарного класса
W_pos = (n_neg / max(1, n_pos)) * 1.2 # вес класса номер 1 (миноритарного)
W = {0:1.0, 1:W_pos}


# функция оценки модели
def eval_model(get_proba, Xval, yval):
    p = get_proba(Xval)
    prec, rec, thr = precision_recall_curve(yval, p)
    f1s = 2*prec[1:]*rec[1:] / (prec[1:]+rec[1:]+1e-12)  # выбираем метрику f1, так как она оринетируется на миноритарный класс
    i = int(np.nanargmax(f1s))
    best_thr, best_f1 = float(thr[i]), float(f1s[i])
    yhat = (p >= best_thr).astype(int)
    P,R,F1,_ = precision_recall_fscore_support(yval, yhat, average="binary", zero_division=0)
    cm = confusion_matrix(yval, yhat)
    return dict(best_thr=best_thr, F1=best_f1, P=P, R=R, cm=cm, probs=p)


# сами модели
# Logistic Regression
lr = LogisticRegression(C=2.0, solver="liblinear", class_weight=W, max_iter=3000)
lr.fit(X_tr, y_tr)
res_lr = eval_model(lambda X: lr.predict_proba(X)[:,1], X_va, y_va)

# LinearSVC + calibration
svc = LinearSVC(class_weight=W, max_iter=5000)
svc_cal = CalibratedClassifierCV(svc, method="sigmoid", cv=5)
svc_cal.fit(X_tr, y_tr)
res_svc = eval_model(lambda X: svc_cal.predict_proba(X)[:,1], X_va, y_va)

# SGD + calibration
sgd = SGDClassifier(loss="modified_huber", alpha=1e-5, max_iter=120, class_weight=W, tol=1e-4)
sgd_cal = CalibratedClassifierCV(sgd, method="sigmoid", cv=5)
sgd_cal.fit(X_tr, y_tr)
res_sgd = eval_model(lambda X: sgd_cal.predict_proba(X)[:,1], X_va, y_va)


summary = pd.DataFrame([
    {"model":"LogReg",         **res_lr},
    {"model":"LinearSVC+Cal",  **res_svc},
    {"model":"SGD(mod_h)+Cal", **res_sgd},
]).sort_values("F1", ascending=False).reset_index(drop=True)


print("=== Validation results ===")
print(summary)
for name,res in [("LogReg",res_lr),("SVC+Cal",res_svc),("SGD+Cal",res_sgd)]:
    print("\n",name,"Confusion:\n",res["cm"])


# выбираем лучшую модель
best_name = summary.loc[0,"model"]
print("\nChosen model:", best_name)

# переобучаем моделю на всем train и test
X_all_tfidf = ct.fit_transform(X_all_df)
meta_all    = csr_matrix(make_meta(train_full["question_raw"]))
X_all       = hstack([X_all_tfidf, meta_all], format='csr')

X_test_df = test_full[["question_lem","question_raw"]]
X_test_tfidf = ct.transform(X_test_df)
meta_test    = csr_matrix(make_meta(test_full["question_raw"]))
X_test       = hstack([X_test_tfidf, meta_test], format='csr')


def make_model(name):
    if name=="LogReg":
        m = LogisticRegression(C=2.0, solver="liblinear", class_weight=W, max_iter=3000)
        proba = lambda X: m.predict_proba(X)[:,1]
    elif name=="LinearSVC+Cal":
        base = LinearSVC(class_weight=W, max_iter=5000)
        m = CalibratedClassifierCV(base, method="sigmoid", cv=5)
        proba = lambda X: m.predict_proba(X)[:,1]
    else:
        base = SGDClassifier(loss="modified_huber", alpha=1e-5, max_iter=120, class_weight=W, tol=1e-4)
        m = CalibratedClassifierCV(base, method="sigmoid", cv=5)
        proba = lambda X: m.predict_proba(X)[:,1]
    return m, proba

final_model, proba_fn = make_model(best_name)
final_model.fit(X_all, y)

# создаем файл submission
probs_test = proba_fn(X_test)

pd.DataFrame({
    "ID": test_full["ID"],
    "score": probs_test
}).to_csv("submission.csv", index=False)

print("\nСохранение файла с метриками: submission.csv (ID, score)")

100%|██████████| 11087/11087 [00:00<00:00, 15549.74it/s]


=== Validation results ===
            model  best_thr        F1         P         R  \
0  SGD(mod_h)+Cal  0.105549  0.262744  0.160441  0.723917   
1          LogReg  0.134451  0.253032  0.152695  0.736758   
2   LinearSVC+Cal  0.093187  0.243697  0.145637  0.744783   

                           cm  \
0  [[3017, 2360], [172, 451]]   
1  [[2830, 2547], [164, 459]]   
2  [[2655, 2722], [159, 464]]   

                                               probs  
0  [0.06732157909824042, 0.08857046864580445, 0.1...  
1  [0.025290563208516605, 0.0413194818116564, 0.0...  
2  [0.07881012712245636, 0.07640378773597503, 0.0...  

 LogReg Confusion:
 [[2830 2547]
 [ 164  459]]

 SVC+Cal Confusion:
 [[2655 2722]
 [ 159  464]]

 SGD+Cal Confusion:
 [[3017 2360]
 [ 172  451]]

Chosen model: SGD(mod_h)+Cal

Сохранение файла с метриками: submission.csv (ID, score)
